In [2]:
import pandas as pd
import numpy as np
from pdfminer.high_level import extract_text
import re
import spacy
from spacy.matcher import Matcher
from nltk.corpus import stopwords

In [ ]:
text = extract_text('resume.pdf')
print(text)

In [ ]:
extracted_text={}

In [ ]:
def get_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

email = get_email_addresses(text)
print(email)

extracted_text["E-Mail"] = email

In [ ]:
def get_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{10})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', num) for num in phone_numbers]

phone_number= get_phone_numbers(text)
print(phone_number)

extracted_text["Phone Number"] = phone_number

In [ ]:
nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

In [ ]:
def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME', [pattern], on_match = None)
    matches = matcher(nlp_text)
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text
name = extract_name(text)
print(name)
extracted_text["Name"] = name

In [ ]:
def extract_skills(resume_text):
    nlp_text = nlp(resume_text)
    tokens = [token.text for token in nlp_text if not token.is_stop]
    skills = ["machine learning",
             "deep learning",
             "nlp",
             "mysql",
             "computer vision",
             "opencv",
             "mongodb",
             "artificial intelligence",
             "ai",
             "flask",
             "robotics",
             "data structures",
             "python",
             "c++",
             "matlab",
             "css",
             "html",
              "Java",
              "Android Development",
              "C"]
    skillset = []
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)

    return [i.capitalize() for i in set([i.lower() for i in skillset])]

skills = []
skills = extract_skills(text)

extracted_text["Skills"] = skills
skills

In [ ]:
STOPWORDS = set(stopwords.words('english'))
EDUCATION = [
            'BE','B.E.', 'B.E', 'BSc.', 'Master of Computer Application',
            'MCA','BTECH', 'B.TECH', 'M.TECH', 'MTECH',
            'HSC', '10th', '+2 science'
        ]
def extract_education(resume_text):
    nlp_text = nlp(resume_text)
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]
    edu = {}
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education
education = extract_education(text)
education
extracted_text["Qualification"] = education

In [ ]:
extracted_text

In [ ]:
sub_patterns = ['[A-Z][a-z]* University','[A-Z][a-z]* Educational Institute',
                'University of [A-Z][a-z]*',
                'Ecole [A-Z][a-z]*']
pattern = '({})'.format('|'.join(sub_patterns))
matches = re.findall(pattern, text)

extracted_text["Institutes"] = matches
matches

In [ ]:
sub_patterns = ['[A-Z][a-z]* [A-Z][a-z]* Private Limited','[A-Z][a-z]* [A-Z][a-z]* Pvt. Ltd.','[A-Z][a-z]* [A-Z][a-z]* Inc.', '[A-Z][a-z]* LLC',
                ]
pattern = '({})'.format('|'.join(sub_patterns))
Exp = re.findall(pattern, text)
extracted_text["Experience"] = Exp
Exp

In [ ]:
extracted_text